Como funciona MLP?
link: https://elcaiseri.medium.com/building-a-multi-layer-perceptron-from-scratch-with-numpy-e4cee82ab06d


In [543]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas

In [544]:
# CHOOSE DATASET

# Binary classification dataset
#diabetes = datasets.fetch_openml("diabetes", version = 1, as_frame = True)

# Regression dataset
diabetes = datasets.load_diabetes( as_frame = True) 

X = diabetes.data.values
y = diabetes.target.values
#for the binary classification
#y = diabetes.target.map({'tested_negative': 0, 'tested_positive': 1}).values


X.shape

(442, 10)

In [545]:
#train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

In [546]:
# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

**Multi-layer perceptron (MLP)**: é um tipo de neurol network que consiste em pelo menos 3 *layers*: **input layer**, uma ou mais **hidden layer** e uma **output layer**. Cada neuronio de uma camada está conectado a todos os neuronios da camada seguinte. O **MLP** aprende ajustando os pesos destas conexões de forma a minimizar o erro das previsões, usando um processo chamado *backpropagation*.

Com **_ init _** definimos os parâmetros que precisamos de modo a torna-los treinaveis. Neste caso temos 6 *layers*, com 5 conexões definidas a baixo, cada uma com 64 neurons.
(No pytorch não declaramos layers mas as conexões entre elas)

Depois com **foward** indicamos a ordem em que o processo irá ocorrer.

------------------------------------
Nota:
- se for uma rede muito grande usa-se, tipicamente o esquema pirâmide, aumenta se gradualmente a dimensão das camadas e depois diminui-se para não passarmos de 1028 neuronios para 1 resultado final.


In [547]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=1, dropout_prob=0.5):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 64)   #input_size: 10 layers (regression); 8 layers (classification)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.out = nn.Linear(64, output_size)
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.out(x)
        return x

Na célula seguinte definimos o valor de:
- **epochs**: nº de vezes que o modelo corre o dataset
- **learning rate**: o valor do passo a cada ciclo
- **dropout**: "desliga"/ zera aleatoriamente alguns neurónios da rede em cada foward pass, reduzindo o overfitting. (neste caso 10% dos neuronios não são tidos em conta durante o treino).
- **batch size**: dimentsão da amostra que iremos treinar de cada vez (no caso 64, dimensão de cada camada)

---------------------------------------
Nota:
- **dropout**: Até 30% é adequado, nunca acima de 50%. 
- neuro network tem muitos parametros sendo importante ter um **lr** lento.

- o numero 64 não é arbitario tem a ver com a capacidade do processador (colocar 40 seria parvo já que o processador suporta 32, 64.... por isso se aguenta 40 aguenta 64) igual racicionio para a dimensão das camadas na neurol network ser 64. Se eu tivesse uma dimensão de 128, e o processador a suportasse podia ter 1 epoch apenas pois de uma só vez todo o data set seria corrido.

In [548]:
num_epochs=250   #100 (inicial)
lr=0.002        #0.0005 (inicial)
dropout=0.1
batch_size=64

Na célula abaixo temos a conversão dos dados para tensores de *PyTorch*. Este passo é necessário para que o modelo possa processar os dados.

Com **TensorDataset** juntamos os tensores criados anteriormente **(Xtr e ytr)** num par (X,y) para que o **DataLoader** saiba que cada *input* corresponde a um *label*.

**DataLoader** cria mini-batches a partir do *dataset* e recebe os seguintes parametros:

- **train_dataset**: objeto *PyTorch* do tipo **TensorDataset** que guarda vários tensores e garante que quando pedes um índice ele devolve o par respondente.
- **batch_size**: define quantos exemplos vão ser usados em cada passo do treino
- **shuffle**: ao ser acionado baralha os dados a cada epoch, evitando que a ordem dos dados influencie o treino (importante para redes neurais)




In [549]:
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.float32)
Xte = torch.tensor(Xte, dtype=torch.float32)
yte = torch.tensor(yte, dtype=torch.float32)

# Wrap Xtr and ytr into a dataset
train_dataset = TensorDataset(Xtr, ytr)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


- **model**: define a rede neural
- **.to(device)**: move o modelo para CPU (processadr) ou GPU (com CUDA da NVIDIA)
- **criterion**: define-se a função perda (mede o erro entra previsão do modelo e o valor real). 
- **otimizer**: define o processo de correção dos pesos para reduzir esse erro. Neste caso o otimizador é o **Adam**, responsável por atualizar os pesos da rede aplicando **backpropagation + gradientes**

In [550]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #só para quem tiver instalado cuda

model = MLP(input_size=Xtr.shape[1], dropout_prob=dropout).to(device)  
#criterion = nn.BCEWithLogitsLoss()  # for binary classification
criterion = nn.MSELoss() #for regression
optimizer = optim.Adam(model.parameters(), lr=lr)

Na célula seguinte começamos por definir o loop principal, o modelo será treinado durante **num_epochs** iterações completas sobre os dados.

**model.train()**: permite ir atualizando e aprimorando os resultados e o algoritmo, sem isso, podemos correr o código as vezes que quisermos, o modelo ficaria igual.

**epoch_loss**: cria a variavel que irá somar as perdas ao longo da época.

Para o segundo loop criado vamos buscar mini-baches de treino (já criados com o **DataLoader**). Este procedimento permite treinar os dados por partes em vez de todos de uma vez, mais eficiente e ajuda na generalização.

**logits**: calcula a saída do modelo para aquele batch (saída bruta da rede antes de ativação)

**loss**: compara a predição **logits** com o valor real **batch_y**, o criterio depende do dataset selecionado

Backpropogation:
- **zero_grad()**: limpa gradientes acumulados da iteraçao anterior
- **loss.backward**: calcula os gradientes via backpropagation
- **optimizer.step()**: atualiza os pesos do modelo com base nos gradientes (usa o algoritmo Adam aqui).

O tensor loss é convertido para float (**loss.item()**) e acumula os valores.
- **avg_loss**: calcula-se a perda média da epoch

O **print** serve para mostrar a evolução, se estiver a descer então o modelo está a aprender

---------------------------------
Nota:
- A iteração ao inicio piora antes de começar a melhorar. Se perto do final voltar a piorar significa que já tivemos treino em excesso

In [551]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_x, batch_y in train_dataloader:
        batch_x = batch_x.to(device)    #move os tensores para o dispositivo definido (cpu)
        batch_y = batch_y.to(device)   

        logits = model(batch_x)
        loss = criterion(logits, batch_y.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/250], Loss: 29985.4229
Epoch [2/250], Loss: 29951.1807
Epoch [3/250], Loss: 28841.7939
Epoch [4/250], Loss: 27261.9596
Epoch [5/250], Loss: 22997.1930
Epoch [6/250], Loss: 14281.0868
Epoch [7/250], Loss: 6773.3678
Epoch [8/250], Loss: 7141.8281
Epoch [9/250], Loss: 4831.8443
Epoch [10/250], Loss: 4962.9900
Epoch [11/250], Loss: 4526.1422
Epoch [12/250], Loss: 4115.5717
Epoch [13/250], Loss: 4037.7600
Epoch [14/250], Loss: 3789.7035
Epoch [15/250], Loss: 3720.9008
Epoch [16/250], Loss: 3625.9245
Epoch [17/250], Loss: 3429.7444
Epoch [18/250], Loss: 3738.9560
Epoch [19/250], Loss: 3532.8769
Epoch [20/250], Loss: 3569.1109
Epoch [21/250], Loss: 3332.0111
Epoch [22/250], Loss: 3360.9196
Epoch [23/250], Loss: 3351.9097
Epoch [24/250], Loss: 3205.1760
Epoch [25/250], Loss: 3448.4743
Epoch [26/250], Loss: 3137.7588
Epoch [27/250], Loss: 3105.6490
Epoch [28/250], Loss: 3229.7970
Epoch [29/250], Loss: 3278.9680
Epoch [30/250], Loss: 3291.3159
Epoch [31/250], Loss: 3153.8383
Epoch [32/2

Nesta ultima célula passamos os dados de teste (**Xte**) pelo modelo **MLP**

Dependendo do dataset (regressão ou classificação binária) temos duas respostas possíveis:

- Dataset de classificação binária:
   - calcula a **accuracy**, ou seja, a proporção de previsões corretas em relação ao **yte**
   - (**y_pred.detach().numpy()>0.5** : converte a previsão contínua do TSK em classe 0 ou 1, com threshold = 0.5)

- Dataset de regressão:
  - calcula o **Mean Square Error** (**MSE**) entre a saída prevista e a saída real
  - (converte **yte** e **y_pred** para numpy arrays, usando **.detach().numpy()**) 


In [552]:
y_pred=model(Xte) #saída prevista pelo modelo para cada amostra teste
#print(f'ACC:{accuracy_score(yte.detach().numpy(),y_pred.detach().numpy()>0.5)}') #classification
print(f'MSE:{mean_squared_error(yte.detach().numpy(),y_pred.detach().numpy())}') #regression

MSE:2961.529052734375


Ao contrario do **TSK** e do **ANFIS**, o **MLP** é uma rede neural "pura" (sem fuzzy). 
- Aprende diretamente os pesos via **backpropagation**
- Não existe interpretabilidade em termos de regras linguísticas
- normalmente obtém maior performance numérica, mas perde em explicabilidade. 
O valor do MSE ser mais elevado com este processo pode ser um sinal de underfitting?